In [ ]:
import geopandas as gpd
from ipywidgets import AppLayout, Dropdown, HBox, interactive_output, Layout, ToggleButton, VBox
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import Point

# Set default settings.
%matplotlib widget

# Turn the pyplot interactive mode off to avoid a duplication of widgets.
plt.ioff()

# Load encoded data.
df_enc = pd.read_csv('./data/data_preprocessed/features.csv.gz', compression='gzip')

# Generate tree coordinates.
df = pd.read_csv('./data/data_raw/nyc_trees/nyc_tree_census_2015.csv.gz', compression='gzip')
nyc_trees = gpd.GeoDataFrame(df, geometry=[Point(coordinates) for coordinates in zip(df['longitude'], df['latitude'])])
nyc_trees.crs = {'init': 'epsg:4326', 'no_defs': True}

# Read in New York boroughs.
nyc_boroughs = gpd.read_file(gpd.datasets.get_path('nybb'))
nyc_boroughs.rename(columns={'BoroName': 'bname'}, inplace=True)
nyc_boroughs = nyc_boroughs.to_crs(epsg=4326)
nyc_boroughs.crs = {'init': 'epsg:4326', 'no_defs': True}

# Read in New York streets.
nyc_streets = gpd.read_file('./data/data_raw/nyc_streets/geo_export_f311eef5-5969-43eb-a781-39fa54d09b70.shp')

# Load classifier.
model = joblib.load('./model/lgbm_clf.pkl')

# Street Tree Questionnaire

In [ ]:
# Create an empty array for gathering answers.
template_answers = pd.Series(index=['steward_Alive',
                                    'steward_Dead|Stump',
                                    'guards_Dead|Stump',
                                    'guards_Harmful',
                                    'guards_Helpful',
                                    'guards_None',
                                    'guards_Unsure',
                                    'sidewalk_Damage',
                                    'sidewalk_Dead|Stump',
                                    'sidewalk_NoDamage',
                                    'root_stone_No',
                                    'root_stone_Yes',
                                    'root_grate_No',
                                    'root_grate_Yes',
                                    'root_other_No',
                                    'root_other_Yes',
                                    'trunk_wire_No',
                                    'trunk_wire_Yes',
                                    'trnk_light_No',
                                    'trnk_light_Yes',
                                    'trnk_other_No',
                                    'trnk_other_Yes',
                                    'brch_light_No',
                                    'brch_light_Yes',
                                    'brch_shoe_No',
                                    'brch_shoe_Yes',
                                    'brch_other_No',
                                    'brch_other_Yes',
                                    'n_neighbors_no neighbor',
                                    'n_neighbors_one neighbor',
                                    'n_neighbors_two or more neighbors'],
                             dtype='int64')

# Creating drop-down widgets for each question.
a = Dropdown(options=[('Please answer', 0),
                      ('There is a steward', 1),
                      ('No steward', 2)],
             description="Is there a steward?",
             layout=Layout(width='650px'),
             style={'description_width': '420px'},
             value=0)

b = Dropdown(options=[('Please answer', 0),
                      ("Don't see a tree", 1),
                      ('Harmful', 2),
                      ('Helpful', 3),
                      ('No guard', 4),
                      ('Unsure', 5)],
             description="If there is a guard, does it impede or support the tree?",
             layout=a.layout,
             style=a.style,
             value=0)

c = Dropdown(options=[('Please answer', 0),
                      ('Damage', 1),
                      ("Don't see a tree", 2),
                      ('No damage', 3)],
             description='In which condition is the sidewalk next to the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

d = Dropdown(options=[('Please answer', 0),
                      ('Stones at the root', 1),
                      ('No stones at the root', 2)],
             description='Are there stones at the root of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)
                      
e = Dropdown(options=[('Please answer', 0),
                      ('Grate at the root', 1),
                      ('No grate at the root', 2)],
             description='Is there a grate at the root of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

f = Dropdown(options=[('Please answer', 0),
                      ('Something else at the root', 1),
                      ('Nothing else at the root', 2)],
             description='Is there something else at the root of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

g = Dropdown(options=[('Please answer', 0),
                      ('Wire touches trunk', 1),
                      ('No wire at trunk', 2)],
             description='Is there a wire at the trunk of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

h = Dropdown(options=[('Please answer', 0),
                      ('Light installations at the trunk', 1),
                      ('No light installations at the trunk', 2)],
             description='Is there a light installation at the trunk of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

i = Dropdown(options=[('Please answer', 0),
                      ('Something else at the trunk', 1),
                      ('Nothing at the trunk', 2)],
             description='Is there something else at the trunk of the tree?',
             layout=a.layout,
             style=a.style,
             value=0)

j = Dropdown(options=[('Please answer', 0),
                      ('Light installations on branches', 1),
                      ('No light installations on branches', 2)],
             description='Are the branches affected by any street light installations?',
             layout=a.layout,
             style=a.style,
             value=0)

k = Dropdown(options=[('Please answer', 0),
                      ('Shoes on branches', 1),
                      ('No shoes on branches', 2)],
             description='Are the branches affected by any shoes?',
             layout=a.layout,
             style=a.style,
             value=0)

l = Dropdown(options=[('Please answer', 0),
                      ('Something else on branches', 1),
                      ('Nothing affects the branches', 2)],
             description='Are the branches affected by anything else?',
             layout=a.layout,
             style=a.style,
             value=0)

m = Dropdown(options=[('Please answer', 0),
                      ('No neighboring tree', 2),
                      ('One neighboring tree', 2),
                      ('More than one neighboring tree', 3)],
             description='How many neighboring street trees are there within a 10m distance?',
             layout=a.layout,
             style=a.style,
             value=0)

def reset_answers(change):
    """Resets the answers to the street tree questionnaire.
    
    :param change: traitlets.utils.bunch.Bunch. Widget emitting change events for button on resetting all answers.

    :return a, b, c, d, e, f, g, h, i, j, k, l, m. ipywidgets.widgets.widget_selection.Dropdown. Drop-down lists to each question.
    """

    a.value = 0
    b.value = 0
    c.value = 0
    d.value = 0
    e.value = 0
    f.value = 0
    g.value = 0
    h.value = 0
    i.value = 0
    j.value = 0
    k.value = 0
    l.value = 0
    m.value = 0
    
    return a, b, c, d, e, f, g, h, i, j, k, l, m


def classify_tree(tree, classifiers):
    """Given visual characteristics of a tree, classifies a street tree into one of three categories and returns a message
    for stdout. This method is called in the method create_tree_quest().
    
    :param tree: numpy.ndarray of shape (1, 31). Contains 1 or 0 for each of the visual characteristics of a street tree.
    :param classifiers: dict. Contains the optimized classifier.

    :return results: numpy.ndarray of shape (1, 3). Predicted class of street tree health condition.
    """

    results = classifiers[list(classifiers.keys())[0]]['grid_classifier'].predict(tree)

    if results.sum() == 0 | results.sum() > 1:
        print("############################  OUTPUT  #############################")
        print("Given the current answers, this tree cannot be classified correctly.")
        print("###################################################################")
    else:
        if results[0][0] == 1:
            print("#########  OUTPUT  #########")
            print("This tree is dead or a stump.")
            print("############################")
        elif results[0][1] == 1:
            print("#####  OUTPUT  #####")
            print("This tree is healthy.")
            print("####################")
        else:
            print("#############  OUTPUT  #################")
            print("This tree is in a poor or fair condition.")
            print("########################################")
            
    return results


def create_tree_quest(q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12, classifiers=model, template=template_answers):
    """Creates a questionnaire for new street trees and calls classify_tree(). Returns the answers to the questionnaire in a
    filled array 'template'.
    
    :param q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12: int. Values of answers selected from drop-down lists on each question.
    :param classifiers: dict. Contains the optimized classifier.
    :param template: pandas.core.series.Series of shape (31,). Empty pandas series for gathering answers.

    :return template.
    """
    
    # Is there a steward?
    if q0 == 1:
        template['steward_Alive'] = 1
        template['steward_Dead|Stump'] = 0
    elif q0 == 2:
        template['steward_Alive'] = 0
        template['steward_Dead|Stump'] = 1
    else:
        template['steward_Alive'] = np.nan
        template['steward_Dead|Stump'] = np.nan
    
    # If there is a guard, does it impede or support the tree?
    if q1 == 1:
        template['guards_Dead|Stump'] = 1
        template['guards_Harmful'] = 0
        template['guards_Helpful'] = 0
        template['guards_None'] = 0
        template['guards_Unsure'] = 0
    elif q1 == 2:
        template['guards_Dead|Stump'] = 0
        template['guards_Harmful'] = 1
        template['guards_Helpful'] = 0
        template['guards_None'] = 0
        template['guards_Unsure'] = 0
    elif q1 == 3:
        template['guards_Dead|Stump'] = 0
        template['guards_Harmful'] = 0
        template['guards_Helpful'] = 1
        template['guards_None'] = 0
        template['guards_Unsure'] = 0
    elif q1 == 4:
        template['guards_Dead|Stump'] = 0
        template['guards_Harmful'] = 0
        template['guards_Helpful'] = 0
        template['guards_None'] = 1
        template['guards_Unsure'] = 0
    elif q1 == 5:
        template['guards_Dead|Stump'] = 0
        template['guards_Harmful'] = 0
        template['guards_Helpful'] = 0
        template['guards_None'] = 0
        template['guards_Unsure'] = 1
    else:
        template['guards_Dead|Stump'] = np.nan
        template['guards_Harmful'] = np.nan
        template['guards_Helpful'] = np.nan
        template['guards_None'] = np.nan
        template['guards_Unsure'] = np.nan
        
    # In which condition is the sidewalk next to the tree?
    if q2 == 1:
        template['sidewalk_Damage'] = 1
        template['sidewalk_Dead|Stump'] = 0
        template['sidewalk_NoDamage'] = 0
    elif q2 == 2:
        template['sidewalk_Damage'] = 0
        template['sidewalk_Dead|Stump'] = 1
        template['sidewalk_NoDamage'] = 0
    elif q2 == 3:
        template['sidewalk_Damage'] = 0
        template['sidewalk_Dead|Stump'] = 0
        template['sidewalk_NoDamage'] = 1
    else:
        template['sidewalk_Damage'] = np.nan
        template['sidewalk_Dead|Stump'] = np.nan
        template['sidewalk_NoDamage'] = np.nan
        
    # Are there stones at the root of the tree?
    if q3 == 1:
        template['root_stone_No'] = 0
        template['root_stone_Yes'] = 1
    elif q3 == 2:
        template['root_stone_No'] = 1
        template['root_stone_Yes'] = 0
    else:
        template['root_stone_No'] = np.nan
        template['root_stone_Yes'] = np.nan    
        
    # Is there a grate at the root of the tree?
    if q4 == 1:
        template['root_grate_No'] = 0
        template['root_grate_Yes'] = 1
    elif q4 == 2:
        template['root_grate_No'] = 1
        template['root_grate_Yes'] = 0
    else:
        template['root_grate_No'] = np.nan
        template['root_grate_Yes'] = np.nan
        
    # Is there something else at the root of the tree?
    if q5 == 1:
        template['root_other_No'] = 0
        template['root_other_Yes'] = 1
    elif q5 == 2:
        template['root_other_No'] = 1
        template['root_other_Yes'] = 0
    else:
        template['root_other_No'] = np.nan
        template['root_other_Yes'] = np.nan
        
    # Is there a wire at the trunk of the tree?
    if q6 == 1:
        template['trunk_wire_No'] = 0
        template['trunk_wire_Yes'] = 1
    elif q6 == 2:
        template['trunk_wire_No'] = 1
        template['trunk_wire_Yes'] = 0
    else:
        template['trunk_wire_No'] = np.nan
        template['trunk_wire_Yes'] = np.nan
        
    # Is there a light installation at the trunk of the tree?
    if q7 == 1:
        template['trnk_light_No'] = 0
        template['trnk_light_Yes'] = 1
    elif q7 == 2:
        template['trnk_light_No'] = 1
        template['trnk_light_Yes'] = 0
    else:
        template['trnk_light_No'] = np.nan
        template['trnk_light_Yes'] = np.nan
        
    # Is there something else at the trunk of the tree?
    if q8 == 1:
        template['trnk_other_No'] = 0
        template['trnk_other_Yes'] = 1
    elif q8 == 2:
        template['trnk_other_No'] = 1
        template['trnk_other_Yes'] = 0
    else:
        template['trnk_other_No'] = np.nan
        template['trnk_other_Yes'] = np.nan

    # Are the branches affected by any street light installations?
    if q9 == 1:
        template['brch_light_No'] = 0
        template['brch_light_Yes'] = 1
    elif q9 == 2:
        template['brch_light_No'] = 1
        template['brch_light_Yes'] = 0
    else:
        template['brch_light_No'] = np.nan
        template['brch_light_Yes'] = np.nan
        
    # Are the branches affected by any shoes?
    if q10 == 1:
        template['brch_shoe_No'] = 0
        template['brch_shoe_Yes'] = 1
    elif q10 == 2:
        template['brch_shoe_No'] = 1
        template['brch_shoe_Yes'] = 0
    else:
        template['brch_shoe_No'] = np.nan
        template['brch_shoe_Yes'] = np.nan
        
    # Are the branches affected by anything else?
    if q11 == 1:
        template['brch_other_No'] = 0
        template['brch_other_Yes'] = 1
    elif q11 == 2:
        template['brch_other_No'] = 1
        template['brch_other_Yes'] = 0
    else:
        template['brch_other_No'] = np.nan
        template['brch_other_Yes'] = np.nan
        
    # How many neighboring street trees are there within 10m from a tree?
    if q12 == 1:
        template['n_neighbors_no neighbor'] = 1
        template['n_neighbors_one neighbor'] = 0
        template['n_neighbors_two or more neighbors'] = 0
    elif q12 == 2:
        template['n_neighbors_no neighbor'] = 0
        template['n_neighbors_one neighbor'] = 1
        template['n_neighbors_two or more neighbors'] = 0
    elif q12 == 3:
        template['n_neighbors_no neighbor'] = 0
        template['n_neighbors_one neighbor'] = 0
        template['n_neighbors_two or more neighbors'] = 1
    else:
        template['n_neighbors_no neighbor'] = np.nan
        template['n_neighbors_one neighbor'] = np.nan
        template['n_neighbors_two or more neighbors'] = np.nan
        
    if template.isnull().any():
        print("#############  OUTPUT  #################")
        print("Please provide answers to all questions.")
        print("########################################")
    else:
        # Classify health condition of street tree.
        query = np.array([template.astype('int64')])
        tree_health = classify_tree(query, classifiers)
    
    return template


# Gather the answers from the dropdown widgets and send to stdout. This is the connection between create_tree_quest() and
# the drop-down lists of the questionnaire.
out = interactive_output(create_tree_quest, {'q0': a,
                                             'q1': b,
                                             'q2': c,
                                             'q3': d,
                                             'q4': e,
                                             'q5': f,
                                             'q6': g,
                                             'q7': h,
                                             'q8': i,
                                             'q9': j,
                                             'q10': k,
                                             'q11': l,
                                             'q12': m})

# Create a widget for framing the questionnaire and the stdout.
questionnaire = HBox([VBox([a, b, c, d, e, f, g, h, i, j, k, l, m]),
                      out])

# Define a button for resetting all answers.
reset_button = ToggleButton(description='Reset answers',
                            layout=Layout(width='150px',
                                          heigth='auto',
                                           margin='right'))

# Make widgets listen to any click-actions by users on the reset_button.
reset_button.observe(reset_answers, 'value')
questionnaire.observe(reset_button)

# Arrange the widgets for this part of the app.
AppLayout(header=None,
          left_sidebar=None,
          center=questionnaire,
          right_sidebar=None,
          footer=reset_button,
          pane_heights=[0, 13, 1],
          grid_gap='20px')

# Street Tree Map: Street trees in New York

In [ ]:
# Configure plot framework.
fig, axs = plt.subplots(figsize=(6.6, 5))
fig.canvas.header_visible = False
axs.axis('off')

# Plot NYC boroughs.
nyc_boroughs.plot(alpha=0.7, ax=axs, color='w', edgecolor='k')
for borough in nyc_boroughs.itertuples(index=True, name='bname'):
    plt.text(borough.geometry.centroid.x,
             borough.geometry.centroid.y,
             borough.bname, 
             bbox={
                 'boxstyle': 'square, pad=0.3',
                 'edgecolor': 'k',
                 'facecolor': 'w'
                 })

# Plot NYC streets for additional context.
nyc_streets.plot(ax=axs, alpha=0.1, color='grey')

def update_map(change, data=df_enc, trees=nyc_trees):
    """Based on the selected value from a filter drop-down list, updates a map with information on the health
    condition of street trees.
    
    :param change: traitlets.utils.bunch.Bunch. Widget emitting change events for drop-down list filter.
    :param data: pandas.core.frame.DataFrame. Contains the encoded data on visual characteristics of street trees.
    :param trees: geopandas.geodataframe.GeoDataFrame. Contains geographical data for plotting street trees.

    :return tree_visual: geopandas.geodataframe.GeoDataFrame. Points of street trees in respective health condition.
    """

    if change['new'] == 2:
        tree_visual = trees.loc[data.loc[data['health_Good'] == 1].index]
        tree_visual.plot(ax=axs, alpha=0.5, color='green', edgecolor=None, markersize=1.0)
    elif change['new'] == 3:
        tree_visual = trees.loc[data.loc[data['health_Dead|Stump'] == 1].index]
        tree_visual.plot(ax=axs, alpha=0.5, color='red', edgecolor=None, markersize=1.0)
    elif change['new'] == 4:
        tree_visual = trees.loc[data.loc[data['health_Poor|Fair'] == 1].index]
        tree_visual.plot(ax=axs, alpha=0.5, color='orange', edgecolor=None, markersize=1.0)
    else:
        tree_visual = trees.plot(ax=axs, alpha=1.0, color='white', edgecolor='white', markersize=1.1)
        fig.canvas.draw()
    print(type(tree_visual))
    return tree_visual


def find_similar_trees(change, answers=template_answers, axis=axs, boroughs=nyc_boroughs, data=df_enc, trees=nyc_trees):
    """Via the cosine similarity of the answers and each of the other street trees, finds and plots ten most similar street trees.
    
    :param change: traitlets.utils.bunch.Bunch. Widget emitting change events for similar_button.
    :param answers: pandas.core.series.Series of shape (31,). Pandas series with answers from street tree questionnaire.
    :param axis: matplotlib.axes._subplots.AxesSubplot. Axes of the plot framework.
    :param boroughs: geopandas.geodataframe.GeoDataFrame. Includes data on the multipolygons of boroughs in New York City.
    :param data: pandas.core.frame.DataFrame. Contains the encoded data on visual characteristics of street trees.
    :param trees: geopandas.geodataframe.GeoDataFrame. Contains geographical data for plotting street trees.

    :return trees_similar: geopandas.geodataframe.GeoDataFrame. Points of ten most similar street trees (similar to answers).
    """
    
    # Redraw NYC boroughs.
    boroughs.plot(alpha=1.0, ax=axis, color='w', edgecolor=None)
    for borough in boroughs.itertuples(index=True, name='bname'):
        plt.text(borough.geometry.centroid.x,
                 borough.geometry.centroid.y,
                 borough.bname, 
                 bbox={
                     'boxstyle': 'square, pad=0.3',
                     'edgecolor': 'k',
                     'facecolor': 'w'
                     })

    change['new'] = answers.values.astype(int)
    
    # Calculate the cosine similarity to find similar entries in the dataset, order by similarity, and select top ten.
    data['similarity'] = np.einsum('i,ji->j', change['new'], data.loc[:, 'steward_Alive':'n_neighbors_two or more neighbors'].values) \
                         / (np.linalg.norm(change['new']) * np.linalg.norm(data.loc[:, 'steward_Alive':'n_neighbors_two or more neighbors'].values))
    top_ten_most_similar_trees = data['similarity'].sort_values(ascending=False)[:10].index
    trees_similar = trees.loc[top_ten_most_similar_trees]
    
    # Visualize top ten similar trees.
    trees_similar.plot(ax=axis, facecolors='none', edgecolors='k', linewidth=1.5)
    
    return trees_similar
    
                         
# Specifiy dropdown menu for displaying trees.
dropdown_list = Dropdown(options=[('All street trees', 1),
                                  ('All healthy street trees', 2),
                                  ('All dead street trees', 3),
                                  ('All poor or fair street trees', 4)],
                         value=1,
                         description='Filter:',
                         layout=Layout(width='50%',
                                       margin='0px 350px 0px 492px',
                                       object_position='top',
                                       object_fit=None))

# Specify button for triggering search for similarity.
similar_button = ToggleButton(button_style='info',
                              description='Show similar trees & clear filter',
                              layout=Layout(width='100%',
                                            margin='527px 0px 0px 0px',
                                            object_position='top',
                                            object_fit=None))

# Make widgets listen to any click-actions by users.
dropdown_list.observe(update_map, 'value')
similar_button.observe(find_similar_trees, 'value')

# Arrange the widgets for this part of the app.
AppLayout(header=None,
          left_sidebar=None,
          center=fig.canvas,
          right_sidebar=similar_button,
          footer=dropdown_list,
          pane_widths=[0, 5, 2],
          pane_heights=[0, 6, 1],
          grid_gap='10px')